In [1]:

from pathlib import Path


import pandas as pd
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, silhouette_score, \
                            homogeneity_completeness_v_measure
from sklearn.metrics.cluster import contingency_matrix
from sklearn.preprocessing import LabelEncoder
import numpy as np



In [2]:
import skmisc

In [4]:
pip install --upgrade pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 39.4 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0


In [3]:
import stlearn as st
import scanpy

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# all 12 samples
sample_list = ["151507", "151508", "151509",
               "151510", "151669", "151670",
               "151671", "151672", "151673",
               "151674", "151675", "151676"]
# cluster number of each sample
cluster_number = [7, 7, 7,
                  7, 5, 5, 
                  5, 5, 7,
                  7, 7, 7]

In [7]:
BASE_PATH = Path("/content/drive/MyDrive/Human_Brain_spatialLIBD")

In [8]:
for i in range(len(sample_list)):

  #load data
  data_PATH = BASE_PATH /"{}".format(sample_list[i])
  data = st.Read10X(data_PATH, count_file = 'filtered_feature_bc_matrix.h5') 

  # pre-processing for gene count table
  st.pp.filter_genes(data,min_cells=1)
  st.pp.normalize_total(data)
  st.pp.log1p(data)

  # get top 2000 hvgs
  scanpy.pp.highly_variable_genes(data, flavor = "seurat_v3", n_top_genes = 2000)

  res = []
  for index in range(len(data.var.highly_variable)):
    if data.var.highly_variable[index]:
      res.append(index)

  data = data[:,res]

  # spot tile is the intermediate result of image pre-processing
  TILE_PATH = Path("/tmp/{}_tiles".format(sample_list[i]))
  TILE_PATH.mkdir(parents=True, exist_ok=True)

  # run PCA for gene expression data
  st.em.run_pca(data,n_comps=15)
  # pre-processing for spot image
  st.pp.tiling(data, TILE_PATH)

  # this step uses deep learning model to extract high-level features from tile images
  # may need few minutes to be completed
  st.pp.extract_feature(data)

  data_SME = data.copy()

  # apply stSME to normalise log transformed data
  # with weights from morphological Similarly and physcial distance
  st.spatial.SME.SME_normalize(data_SME, use_data="raw",
                              weights="weights_matrix_pd_md")
  data_SME.X = data_SME.obsm['raw_SME_normalized']

  # K-means clustering on stSME normalised PCA
  st.tl.clustering.kmeans(data_SME, n_clusters = cluster_number[i],  use_data="X_pca", key_added="X_pca_kmeans")
 
  # save cluster resluts
  df = pd.DataFrame(data_SME.obs)
  df.to_csv("/content/drive/MyDrive/Human_Brain_spatialLIBD/{}.stLearn_filtered_markers.csv".format(sample_list[i]))



/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Normalization step is finished in adata.X
Log transformation step is finished in adata.X


/usr/local/lib/python3.10/dist-packages/scanpy/preprocessing/_highly_variable_genes.py:62: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(


PCA is done! Generated in adata.obsm['X_pca'], adata.uns['pca'] and adata.varm['PCs']


Tiling image: 100%|██████████ [ time left: 00:00 ]


...

The morphology feature is added to adata.obsm['X_morphology']!


Adjusting data: 100%|██████████ [ time left: 00:00 ]


The data adjusted by SME is added to adata.obsm['raw_SME_normalized']
Applying Kmeans cluster ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1366: FutureWarning: algorithm='auto' is deprecated, it will be removed in 1.3. Using 'lloyd' instead.
  warnings.warn(


Kmeans cluster is done! The labels are stored in adata.obs["kmeans"]
